<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/Image-Retrieval/Auto_capturing_Images_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import base64
from IPython.display import display, Javascript
from google.colab import output

# Function to capture and upload frames
def capture_and_upload(num_frames=60, duration=10):
    """Captures frames from webcam and uploads them."""

    js_code = f"""
        const start_time = Date.now();
        const end_time = start_time + {duration} * 1000;
        let count = 0;

        async function captureFrame() {{
            try {{
                const stream = await navigator.mediaDevices.getUserMedia({{ video: true }});
                const video = document.createElement('video');
                video.srcObject = stream;
                video.play();

                const canvas = document.createElement('canvas');
                const ctx = canvas.getContext('2d');

                video.onloadedmetadata = () => {{
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;

                    const captureInterval = setInterval(() => {{
                        if (Date.now() > end_time || count >= {num_frames}) {{
                            clearInterval(captureInterval);
                            stream.getTracks().forEach(track => track.stop());
                            return;
                        }}

                        ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                        const dataURL = canvas.toDataURL('image/jpeg', 0.8);

                        const blob = dataURLtoBlob(dataURL);
                        const filename = `frame_${{count}}.jpeg`;

                        uploadFile(blob, filename).then(() => count++);
                    }}, 1000 / ({num_frames} / {duration}));
                }};
            }} catch (error) {{
                console.error("Error accessing camera:", error);
                alert("Could not access the webcam. Ensure permissions are granted.");
            }}
        }}

        function dataURLtoBlob(dataURL) {{
            const [meta, base64] = dataURL.split(';base64,');
            const contentType = meta.split(':')[1];
            const byteString = atob(base64);
            const ab = new ArrayBuffer(byteString.length);
            const ia = new Uint8Array(ab);
            for (let i = 0; i < byteString.length; i++) {{
                ia[i] = byteString.charCodeAt(i);
            }}
            return new Blob([ab], {{ type: contentType }});
        }}

        async function uploadFile(blob, filename) {{
            const reader = new FileReader();
            reader.readAsDataURL(blob);
            reader.onloadend = async () => {{
                const base64String = reader.result.split(',')[1];
                await google.colab.kernel.invokeFunction('upload_to_google', [base64String, filename]);
            }};
        }}

        captureFrame();
    """

    display(Javascript(js_code))
    time.sleep(duration + 2)
    print("Finished capturing frames.")

# Backend function to save uploaded files
def upload_to_google(base64_string, filename):
    output_folder = "processed_images"
    os.makedirs(output_folder, exist_ok=True)
    decoded_image = base64.b64decode(base64_string)
    output_path = os.path.join(output_folder, filename)
    with open(output_path, 'wb') as f:
        f.write(decoded_image)
    print(f"Saved: {output_path}")
    return True

# Register callback
output.register_callback('upload_to_google', upload_to_google)

# Create output folder
os.makedirs("processed_images", exist_ok=True)

# Example: Capture 6 frames in 10 seconds
capture_and_upload(num_frames=6, duration=10)


Update for taking pictures for 1min by every 9 mins

In [ ]:
import os
import time
import base64
from IPython.display import display, Javascript
from google.colab import output

# Function to capture and upload frames periodically
def capture_and_upload(num_frames=60, capture_duration=60, interval_minutes=9):
    """Captures frames from webcam once every interval_minutes for capture_duration seconds."""

    interval_ms = interval_minutes * 60 * 1000  # Convert minutes to milliseconds

    js_code = f"""
        async function startCaptureCycle() {{
            while (true) {{
                console.log("Starting capture session");
                await captureFrames();
                console.log("Waiting {interval_minutes} minutes for the next capture session");
                await new Promise(resolve => setTimeout(resolve, {interval_ms}));
            }}
        }}

        async function captureFrames() {{
            const start_time = Date.now();
            const end_time = start_time + {capture_duration} * 1000;
            let count = 0;

            try {{
                const stream = await navigator.mediaDevices.getUserMedia({{ video: true }});
                const video = document.createElement('video');
                video.srcObject = stream;
                video.play();

                const canvas = document.createElement('canvas');
                const ctx = canvas.getContext('2d');

                video.onloadedmetadata = () => {{
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;

                    const captureInterval = setInterval(() => {{
                        if (Date.now() > end_time || count >= {num_frames}) {{
                            clearInterval(captureInterval);
                            stream.getTracks().forEach(track => track.stop());
                            return;
                        }}

                        ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
                        const dataURL = canvas.toDataURL('image/jpeg', 0.8);
                        const blob = dataURLtoBlob(dataURL);
                        const filename = `frame_${{Date.now()}}.jpeg`;

                        uploadFile(blob, filename).then(() => count++);
                    }}, 1000 / ({num_frames} / {capture_duration}));
                }};
            }} catch (error) {{
                console.error("Error accessing camera:", error);
                alert("Could not access the webcam. Ensure permissions are granted.");
            }}
        }}

        function dataURLtoBlob(dataURL) {{
            const [meta, base64] = dataURL.split(';base64,');
            const contentType = meta.split(':')[1];
            const byteString = atob(base64);
            const ab = new ArrayBuffer(byteString.length);
            const ia = new Uint8Array(ab);
            for (let i = 0; i < byteString.length; i++) {{
                ia[i] = byteString.charCodeAt(i);
            }}
            return new Blob([ab], {{ type: contentType }});
        }}

        async function uploadFile(blob, filename) {{
            const reader = new FileReader();
            reader.readAsDataURL(blob);
            reader.onloadend = async () => {{
                const base64String = reader.result.split(',')[1];
                await google.colab.kernel.invokeFunction('upload_to_google', [base64String, filename]);
            }};
        }}

        startCaptureCycle();
    """

    display(Javascript(js_code))
    print(f"Scheduled webcam capture every {interval_minutes} minutes, recording for {capture_duration} seconds each time.")

# Backend function to save uploaded files
def upload_to_google(base64_string, filename):
    output_folder = "processed_images"
    os.makedirs(output_folder, exist_ok=True)
    decoded_image = base64.b64decode(base64_string)
    output_path = os.path.join(output_folder, filename)
    with open(output_path, 'wb') as f:
        f.write(decoded_image)
    print(f"Saved: {output_path}")
    return True

# Register callback
output.register_callback('upload_to_google', upload_to_google)

# Create output folder
os.makedirs("processed_images", exist_ok=True)

# Example: Capture 25 frames per session, every 9 minutes, for 1 minute
tcapture_and_upload(num_frames=25, capture_duration=60, interval_minutes=9)


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Path to the folder containing captured images
output_folder = "processed_images"

# List all files in the output folder
image_files = sorted([f for f in os.listdir(output_folder) if f.endswith('.jpeg')])

# Display each image
for image_file in image_files:
    image_path = os.path.join(output_folder, image_file)
    img = Image.open(image_path)

    # Display the image
    plt.figure()
    plt.imshow(img)
    plt.axis('off')
    plt.title(image_file)
    plt.show()


For further Studies

In [ ]:
import os
import asyncio
from aiohttp import web
import base64
from PIL import Image
import matplotlib.pyplot as plt

# Path to save processed images
OUTPUT_FOLDER = "processed_images"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

async def handle_frame_upload(request):
    """Handles frame uploads from mobile devices."""
    try:
        # Parse JSON data from the request
        data = await request.json()
        device_id = data.get('device_id', 'unknown_device')
        frame_base64 = data['frame']
        filename = data['filename']

        # Decode the base64 frame and save it
        decoded_image = base64.b64decode(frame_base64)
        device_folder = os.path.join(OUTPUT_FOLDER, device_id)
        os.makedirs(device_folder, exist_ok=True)
        output_path = os.path.join(device_folder, filename)
        with open(output_path, 'wb') as f:
            f.write(decoded_image)
        print(f"Saved: {output_path}")

        return web.Response(text="Frame received successfully.")
    except Exception as e:
        print(f"Error handling frame upload: {e}")
        return web.Response(text="Error processing the frame.", status=500)

async def start_server():
    """Starts an HTTP server for receiving frames."""
    app = web.Application()
    app.router.add_post('/upload_frame', handle_frame_upload)
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, '0.0.0.0', 8080)
    print("Server started on http://0.0.0.0:8080")
    await site.start()

def display_saved_images():
    """Displays all saved images from each device."""
    for device_id in os.listdir(OUTPUT_FOLDER):
        device_folder = os.path.join(OUTPUT_FOLDER, device_id)
        if os.path.isdir(device_folder):
            print(f"Displaying images from device: {device_id}")
            image_files = sorted(
                [f for f in os.listdir(device_folder) if f.endswith('.jpeg')]
            )
            for image_file in image_files:
                image_path = os.path.join(device_folder, image_file)
                img = Image.open(image_path)
                plt.figure()
                plt.imshow(img)
                plt.axis('off')
                plt.title(f"{device_id} - {image_file}")
                plt.show()

if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(start_server())
    except KeyboardInterrupt:
        print("Server stopped.")


web app part react

In [ ]:
import React, { useState, useEffect } from 'react';
import { View, Text, Button, StyleSheet, TextInput, Alert } from 'react-native';
import { Camera, useCameraDevices } from 'react-native-vision-camera';
import axios from 'axios';

export default function App() {
  const [deviceId, setDeviceId] = useState('');
  const [isStreaming, setIsStreaming] = useState(false);
  const devices = useCameraDevices();
  const camera = devices.back;

  useEffect(() => {
    // Request camera permissions
    (async () => {
      const status = await Camera.requestCameraPermission();
      if (status !== 'authorized') {
        Alert.alert('Permission Denied', 'Camera permissions are required to use this app.');
      }
    })();
  }, []);

  const startStreaming = async () => {
    if (!deviceId.trim()) {
      Alert.alert('Error', 'Please enter a unique Device ID.');
      return;
    }

    if (!camera) {
      Alert.alert('Error', 'No camera detected.');
      return;
    }

    setIsStreaming(true);

    try {
      const stream = await Camera.startStream(camera, async (frame) => {
        const base64Image = frame.toBase64(); // Assuming frame can be converted to base64
        const filename = `frame_${Date.now()}.jpeg`;

        // Upload frame to the server
        try {
          await axios.post('http://<server_ip>:8080/upload_frame', {
            device_id: deviceId,
            frame: base64Image,
            filename: filename,
          });
        } catch (error) {
          console.error('Error uploading frame:', error.message);
        }
      });

      // Automatically stop streaming after 10 seconds
      setTimeout(() => {
        Camera.stopStream(stream);
        setIsStreaming(false);
      }, 10000);
    } catch (error) {
      console.error('Error starting camera stream:', error);
      Alert.alert('Error', 'Failed to start streaming.');
      setIsStreaming(false);
    }
  };

  return (
    <View style={styles.container}>
      <Text style={styles.title}>Mobile Camera Streaming</Text>
      <TextInput
        style={styles.input}
        placeholder="Enter Unique Device ID"
        value={deviceId}
        onChangeText={setDeviceId}
      />
      <Button title={isStreaming ? 'Streaming...' : 'Start Streaming'} onPress={startStreaming} disabled={isStreaming} />
      <Text style={styles.info}>Server must be running at http://<server_ip>:8080</Text>
    </View>
  );
}

const styles = StyleSheet.create({
  container: {
    flex: 1,
    justifyContent: 'center',
    alignItems: 'center',
    padding: 16,
    backgroundColor: '#f5f5f5',
  },
  title: {
    fontSize: 24,
    fontWeight: 'bold',
    marginBottom: 20,
  },
  input: {
    width: '80%',
    padding: 10,
    borderWidth: 1,
    borderColor: '#ccc',
    borderRadius: 5,
    marginBottom: 20,
  },
  info: {
    marginTop: 20,
    color: '#666',
  },
});


Using web page

In [ ]:
<!DOCTYPE html>
<html>
  <head>
    <title>Mobile Camera Uploader</title>
  </head>
  <body>
    <h1>Mobile Camera Uploader</h1>
    <p>Ensure your server is running at <strong>http://<server_ip>:8080</strong>.</p>
    <button onclick="startStreaming()">Start Streaming</button>

    <script>
      async function startStreaming() {
        const deviceId = prompt("Enter a unique device ID:");
        const serverUrl = "http://<server_ip>:8080/upload_frame";

        try {
          const stream = await navigator.mediaDevices.getUserMedia({ video: true });
          const video = document.createElement("video");
          video.srcObject = stream;
          video.play();

          const canvas = document.createElement("canvas");
          const ctx = canvas.getContext("2d");

          setInterval(() => {
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
            const dataURL = canvas.toDataURL("image/jpeg", 0.8);

            // Send frame to the server
            fetch(serverUrl, {
              method: "POST",
              headers: { "Content-Type": "application/json" },
              body: JSON.stringify({
                device_id: deviceId,
                frame: dataURL.split(",")[1], // Remove 'data:image/jpeg;base64,' prefix
                filename: `frame_${Date.now()}.jpeg`,
              }),
            })
              .then((response) => response.text())
              .then((text) => console.log(text))
              .catch((err) => console.error("Error uploading frame:", err));
          }, 1000); // Capture and send a frame every second
        } catch (error) {
          console.error("Error accessing the camera:", error);
          alert("Could not access the camera. Ensure permissions are granted.");
        }
      }
    </script>
  </body>
</html>
